In [ ]:

# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
FOLDERNAME =  "University\ of\ Cyprus/MSc\ Artificial\ Intelligence/NLP-Project"
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
%cd /content/drive/My\ Drive/$FOLDERNAME/
!ls

Mounted at /content/drive
/content/drive/My Drive/University of Cyprus/MSc Artificial Intelligence/NLP-Project
cardiffnlp	    Finetune.ipynb  lstm_model.py  preprocessing.py
data		    finetune.py     Main.ipynb	   __pycache__
DataAnalysis.ipynb  GPT-2.ipynb     main.py	   slangdict.pickle


In [ ]:
!pip install transformers datasets pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, RobertaTokenizerFast
from transformers import Trainer, TrainingArguments,  EarlyStoppingCallback

import datasets
import pandas as pd

from datasets import Dataset, DatasetDict

# Load the dataset
df = pd.read_csv("data/finacial_sentiment_data.csv")

def labels_dist():
    return df['labels'].value_counts()


labels_map = {"negative":0, "neutral":1, "positive":2}

df["Sentiment"] = df['Sentiment'].map(lambda q: labels_map[q])

df = df.rename(columns={"Sentence":"text", "Sentiment":"labels"})

minority_samples = df['labels'].value_counts().values[-1]

#df = df[df['labels'] != 1]

df_pos = df[df['labels'] == 2].sample(minority_samples)
df_neu = df[df['labels'] == 1].sample(minority_samples)
df_neg = df[df['labels'] == 0]

df = pd.concat([df_pos, df_neu, df_neg])
df = df.sample(frac=1)
print(df.head())

print(df['text'][0])

print("Data shape:", df.shape)


train_fac = 0.8
df_train = df.sample(frac=train_fac)
df_test = df.drop(df_train.index)

print("Train shape:", df_train.shape)
print("Test shape:", df_test.shape)

print("# of Na: \n", df.isna().sum())

#ds_encoded.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])




def train():
    trainer.train()

# Evaluate the model
def evaluate():
    trainer.evaluate(ds_encoded["test"])


def save():
    return
    #trainer.save_model(MODEL)
    #model.save_pretrained(MODEL)


                                                   text  labels
5541  In the first nine months of 2010 , the company...       2
3093  A broad range of connectivity options includin...       1
2     For the last quarter of 2010 , Componenta 's n...       2
2298  ALERT: New RSI signal created on $COST - Buy a...       2
3520                                         Long $PCLN       2
The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform , location relevant multimedia content and a new and powerful commercial model .
Data shape: (2580, 2)
Train shape: (2064, 2)
Test shape: (516, 2)
# of Na: 
 text      0
labels    0
dtype: int64


860


In [ ]:
# Load the pre-trained model and tokenizer
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

#tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=3)

In [ ]:

#Dataset fitting preparing
ds_train = Dataset.from_pandas(df_train)
ds_test  = Dataset.from_pandas(df_test)

ds = DatasetDict()

ds['train'] = ds_train
ds['test']  = ds_test

###############################################
# There exist a variety of BERT-based models  #
# that can be fine-tuned. We will go with the #
# basic `bert-base-uncased`.                  #
###############################################

# model_name = "bert-base-uncased"
max_length = 512

# Prepare the data
def tokenize_function(examples):
    return tokenizer(examples['text'],
                      padding=True,
                      truncation=True,
                      max_length=max_length)

ds_encoded = ds.map(tokenize_function, batched=True,
                     batch_size = None)

for c in ds_encoded["train"].column_names: print('-', c)

import torch

from transformers import AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

num_labels = len(df['labels'].unique())

print("Num Labels:", num_labels)

model.to(device)


Map:   0%|          | 0/2064 [00:00<?, ? examples/s]

Map:   0%|          | 0/516 [00:00<?, ? examples/s]

- text
- labels
- __index_level_0__
- input_ids
- attention_mask
cuda
Num Labels: 3


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:

# Fine-tune the model
batch_size = 4

logging_steps = len(ds_encoded["train"]) // batch_size

training_args = TrainingArguments(
    output_dir             = f"{MODEL}-financial",
    weight_decay           = 0.025,
    learning_rate          = 2e-5,
    evaluation_strategy    = "epoch",
    save_strategy          = 'epoch',
    num_train_epochs       = 2,
    logging_steps          = logging_steps,
    load_best_model_at_end = True,
    save_total_limit       = 2,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = 2,
    gradient_accumulation_steps = 8
)

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(pred):
    
    labels = pred.label_ids
    preds  = pred.predictions.argmax(-1)
    
    f1 = f1_score       (labels, preds, average="weighted")
    ac = accuracy_score (labels, preds)
    pr = precision_score(labels, preds, average="weighted")
    re = recall_score   (labels, preds, average="weighted")
    
    return {"Accuracy": ac, "Precision": pr, "Recall": re, "F1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics = compute_metrics,
    train_dataset   = ds_encoded["train"],
    eval_dataset    = ds_encoded["test"],
    
    callbacks       = [EarlyStoppingCallback(early_stopping_patience = 3)], 
    
    tokenizer       = tokenizer
)


In [ ]:
labels_dist()

2    860
1    860
0    860
Name: labels, dtype: int64

In [ ]:
train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.449812,0.835271,0.838989,0.835271,0.833322
1,No log,0.449896,0.825581,0.826766,0.825581,0.823518


In [ ]:
evaluate()

In [ ]:
save()